# Using Poloniex Steaming API

* Key and Token creation

In [21]:
polo = poloniex('YS7B3GM0-FKGUI63B-3FWQREZD-4WZB4XBS', 
                         'd6cd713ebab7a174aa41fb7a13d2b2c238ce851c1e896e79f1280e27765340be1790f562e647411b442bb0759cf8a1e4a070856a09bc93ba433f175b72050d37')

In [83]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)
print dateToUNIX('2014-02-10 04:23:23')
print dateToUNIX('2014-02-10 01:19:37')
funTester = polo.returnTradeHistoryByTime('BTC_ETH', '2014-02-10 01:19:37', '2014-02-10 04:23:23')
print(funTester)
'https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_ETH&start=1392013177&end=1392013177'

1392024203
1392013177
None


# returnTransferResultsByTime
* find correct library function to transform Datetime tupil to a UNIX time
* cannot request a window larger than one year
* Poloniex can handle up to 30,000 transactions for second, but can't go back in time based on number of transactions

In [74]:
import urllib
import urllib2
import json
import time
import hmac,hashlib

def createTimeStamp(datestr, format="%Y-%m-%d %H:%M:%S"):
    return time.mktime(time.strptime(datestr, format))

class poloniex:
    def __init__(self, APIKey, Secret):
        self.APIKey = APIKey
        self.Secret = Secret

    def post_process(self, before):
        after = before

        # Add timestamps if there isnt one but is a datetime
        if('return' in after):
            if(isinstance(after['return'], list)):
                for x in xrange(0, len(after['return'])):
                    if(isinstance(after['return'][x], dict)):
                        if('datetime' in after['return'][x] and 'timestamp' not in after['return'][x]):
                            after['return'][x]['timestamp'] = float(createTimeStamp(after['return'][x]['datetime']))
                            
        return after

    def api_query(self, command, req={}):

        if(command == "returnTicker" or command == "returnTradeHistory"):
            ret = urllib2.urlopen(urllib2.Request('https://poloniex.com/public?command=' + command))
            return json.loads(ret.read())
        elif(command == "return24Volume"):
            command = "return24hVolume"
            ret = urllib2.urlopen(urllib2.Request('https://poloniex.com/public?command=' + command))
            return json.loads(ret.read())
        elif(command == "returnOrderBook"):
            ret = urllib2.urlopen(urllib2.Request('https://poloniex.com/public?command=' + command + '&currencyPair=' + str(req['currencyPair'])))
            return json.loads(ret.read())
        elif(command == "returnMarketTradeHistory"):
            # should be including starting and ending datetimes  
            # https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_NXT&start=1410158341&end=1410499372
            ret = urllib2.urlopen(urllib2.Request('https://poloniex.com/public?command=' + "returnTradeHistory" + '&currencyPair=' + str(req['currencyPair'])))
            return json.loads(ret.read())
        else:
            req['command'] = command
            req['nonce'] = int(time.time()*1000)
            post_data = urllib.urlencode(req)

            sign = hmac.new(self.Secret, post_data, hashlib.sha512).hexdigest()
            headers = {
                'Sign': sign,
                'Key': self.APIKey
            }

            ret = urllib2.urlopen(urllib2.Request('https://poloniex.com/tradingApi', post_data, headers))
            jsonRet = json.loads(ret.read())
            return self.post_process(jsonRet)


    def returnTicker(self):
        return self.api_query("returnTicker")

    def return24Volume(self):
        return self.api_query("return24Volume")

    def returnOrderBook (self, currencyPair):
        return self.api_query("returnOrderBook", {'currencyPair': currencyPair})

    def returnMarketTradeHistory (self, currencyPair):
        return self.api_query("returnMarketTradeHistory", {'currencyPair': currencyPair})
    

    # Returns all of your balances.
    # Outputs: 
    # {"BTC":"0.59098578","LTC":"3.31117268", ... }
    def returnBalances(self):
        return self.api_query('returnBalances')

    # Returns your open orders for a given market, specified by the "currencyPair" POST parameter, e.g. "BTC_XCP"
    # Inputs:
    # currencyPair  The currency pair e.g. "BTC_XCP"
    # Outputs: 
    # orderNumber   The order number
    # type          sell or buy
    # rate          Price the order is selling or buying at
    # Amount        Quantity of order
    # total         Total value of order (price * quantity)
    def returnOpenOrders(self,currencyPair):
        return self.api_query('returnOpenOrders',{"currencyPair":currencyPair})


    # Returns your trade history for a given market, specified by the "currencyPair" POST parameter
    # Inputs:
    # currencyPair  The currency pair e.g. "BTC_XCP"
    # Outputs: 
    # date          Date in the form: "2014-02-19 03:44:59"
    # rate          Price the order is selling or buying at
    # amount        Quantity of order
    # total         Total value of order (price * quantity)
    # type          sell or buy
    def returnTradeHistory(self,currencyPair):
        return self.api_query('returnTradeHistory',{"currencyPair":currencyPair})
    
    def returnTradeHistoryByTime (self, currencyPair, start, end):
        ret = urllib2.urlopen(urllib2.Request('https://poloniex.com/public?command=returnTradeHistory&currencyPair=' + str(currencyPair) + '&start=' + str(dateToUNIX(start)) + '&end=' + str(dateToUNIX(end))))
        return json.loads(ret.read())
        
    # Places a buy order in a given market. Required POST parameters are "currencyPair", "rate", and "amount". If successful, the method will return the order number.
    # Inputs:
    # currencyPair  The curreny pair
    # rate          price the order is buying at
    # amount        Amount of coins to buy
    # Outputs: 
    # orderNumber   The order number
    def buy(self,currencyPair,rate,amount):
        return self.api_query('buy',{"currencyPair":currencyPair,"rate":rate,"amount":amount})

    # Places a sell order in a given market. Required POST parameters are "currencyPair", "rate", and "amount". If successful, the method will return the order number.
    # Inputs:
    # currencyPair  The curreny pair
    # rate          price the order is selling at
    # amount        Amount of coins to sell
    # Outputs: 
    # orderNumber   The order number
    def sell(self,currencyPair,rate,amount):
        return self.api_query('sell',{"currencyPair":currencyPair,"rate":rate,"amount":amount})

    # Cancels an order you have placed in a given market. Required POST parameters are "currencyPair" and "orderNumber".
    # Inputs:
    # currencyPair  The curreny pair
    # orderNumber   The order number to cancel
    # Outputs: 
    # succes        1 or 0
    def cancel(self,currencyPair,orderNumber):
        return self.api_query('cancelOrder',{"currencyPair":currencyPair,"orderNumber":orderNumber})

    # Immediately places a withdrawal for a given currency, with no email confirmation. In order to use this method, the withdrawal privilege must be enabled for your API key. Required POST parameters are "currency", "amount", and "address". Sample output: {"response":"Withdrew 2398 NXT."} 
    # Inputs:
    # currency      The currency to withdraw
    # amount        The amount of this coin to withdraw
    # address       The withdrawal address
    # Outputs: 
    # response      Text containing message about the withdrawal
    def withdraw(self, currency, amount, address):
        return self.api_query('withdraw',{"currency":currency, "amount":amount, "address":address})

In [45]:
# converts the date object attribute of the JSON to UNIX format 
import time
import datetime
def dateToUNIX(date):
    date = str(date)
    year = int(date[:4])
    month = int(date[5:-12])
    day = int(date[8:-9])
    hour = int(date[11:-6])
    minute = int(date[14:-3])
    second = int(date[17:])
    dt = datetime.datetime(year, month, day, hour, minute, second, 0)
    UNIX = time.mktime(dt.timetuple())
    UNIX = str(UNIX)[:-2]
    return UNIX
print dateToUNIX('2017-04-20 12:54:13')

1492707253


* Handling rule of maximum of 6 requests per second so ip doesn't get banned from public API
* Also cannot fetch "absurdly large amounts of data"

In [10]:
from poloniex import Poloniex, Coach
myCoach = Coach()

public = Poloniex(coach=myCoach)
private = Poloniex(polo.Key, polo.Secret, coach=myCoach)
# now make calls using both 'private' and 'public' and myCoach will handle both